In [1]:
import sqlite3
import pandas as pd

In [2]:
CHEMBL_DB_PATH = "../raw-data/chembldb/chembl_35/chembl_35_sqlite/chembl_35.db"

In [5]:
# Establish a connection to the DB and a cursor
con = sqlite3.connect(CHEMBL_DB_PATH)
cur = con.cursor()

In [32]:
res = cur.execute(
    """
    SELECT comp.molregno, comp.canonical_smiles, mol.indication_class, rec.compound_name, rec.record_id, ind.mesh_heading 
    FROM compound_structures as comp
    INNER JOIN molecule_dictionary as mol
    ON comp.molregno = mol.molregno
    INNER JOIN compound_records as rec
    on mol.molregno = rec.molregno
    INNER JOIN drug_indication as ind
    ON rec.record_id = ind.record_id
    ORDER BY comp.molregno
    """
)

In [33]:
for i, r in enumerate(res):
    print(r)
    if i > 10:
        break

(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Hypertension')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Stress Disorders, Post-Traumatic')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Tobacco Use Disorder')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Amphetamine-Related Disorders')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Urinary Bladder, Neurogenic')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Anxiety')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Kidney Calculi')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC', 'Antihypertensive', 'PRAZOSIN', 2791601, 'Substance Withdrawal Syndrome')
(97, 'COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3